In [3]:
#   Shobhit Ranjan
#   Simple Particle Swarm Optimization (PSO) Parallelization Test


from __future__ import division
import random
import math

#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)
def func1(x):
    total=0
    for i in range(len(x)):
        total+=x[i]**2
    return total


class Particle:
    def __init__(self,x0):
        self.position_i=[]       # particle position
        self.velocity_i=[]       # particle velocity
        self.pos_best_i=[]       # best position individual
        self.err_best_i=-1       # best error individual
        self.err_i=-1            # error individual
        self.trial1 = 0
        self.trial2 = 0
        self.trial3 = 0

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5     # constant inertia weight (how much to weigh the previous velocity)
        c1=1      # cognative constant
        c2=2      # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        self.err_best_g=-1            # best error for group
        self.pos_best_g=[]            # best position for group
        self.trial4 = 0
        # establish the swarm
        swarm=[]
        self.sw=swarm
        self.n = num_particles
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < self.err_best_g or self.err_best_g == -1:
                    self.pos_best_g=list(swarm[j].position_i)
                    self.err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(self.pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

    def evolve(self):
        self.bruh()
        return self.pos_best_g,self.err_best_g
    
    def bruh(self):
        for i in range(0,self.n):
            self.sw[i].trial1 = 1
            self.sw[i].trial2 = 2
            self.sw[i].trial3 = 3
            self.trial4 = 4
            
    def fin(self):
        self.evolve()
        for i in range(0,self.n):
            print (self.sw[i].trial1, self.sw[i].trial2, self.sw[i].trial3,self.trial4 )
    def nofin(self):
        self.evolve()
        for i in range(0,self.n):
            self.sw[i].trial1 = 10
            self.sw[i].trial2 = 20
            self.sw[i].trial3 = 30
            self.trial4 = 40
        for i in range(0,self.n):
            print (self.sw[i].trial1, self.sw[i].trial2, self.sw[i].trial3,self.trial4 )
            

In [4]:
#--- TEST RUN ----------------------------------------------------------------------+

if __name__ == "__PSO__":
    main()

initial=[5,5]               # initial starting location [x1,x2...]
bounds=[(-10,10),(-10,10)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
k = PSO(func1,initial,bounds,num_particles=15,maxiter=30)
a,b = k.evolve()
print(a,b)

[0.0009331896666508825, -0.0006919642338911287] 1.3496574549285218e-06


In [5]:
#-------CHECK NUMBER OF AVAILABLE CPUs-----------------------------------------------+
import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  8


In [6]:
#---------FUNCTION TO HANDLE INPUTS TO THE SWARM CLASS--------------------------------+
import time
def play(num_particles):
    maxiter = num_particles
    #noof_swarms = 2
    pos = []
    err = []
    #for i in range(noof_swarms):
    k = PSO(func1,initial,bounds,num_particles,maxiter)
    a, b = k.evolve()
    pos.append(a)
    err.append(b)
    return (pos,err)
    #k.fin()

# Single Core Implementation

In [8]:
#----------SINGLE CORE IMPLEMENTATION----------------------------------------------------+
import time
start = time.time()   #start time

num_particles=1000                                    
noof_swarms=4
for i in range(noof_swarms):
    play(num_particles)
    
end = time.time()     #end time
ev1 = end - start     #time taken


print("Single core, 4 swarms:",ev1,"seconds")

Single core, 4 swarms: 13.129693984985352 seconds


# Multicore implementation

In [11]:
#----------MULTIPLE CORE (4 CORE) IMPLEMENTATION-----------------------------------------+
import multiprocessing
import threading
import time

start = time.time() #start time


noof_swarms = 4
num_particles=1000
proc = []
for i in range(noof_swarms):
    pp = multiprocessing.Process(target = play, args=(num_particles,))
    #pp = threading.Thread(target = play, args=(1500,1500))
    proc.append(pp)
for k in proc:
    k.start()
    #print("process:",k)
for k in proc:
    k.join()
    
    
end = time.time()  #end time
ev2 = end - start  #time taken 

print("4 cores, 4 swarms:",ev2,"seconds")

4 cores, 4 swarms: 4.277662992477417 seconds


# Multithread implementation

In [13]:
#-----------MULTI THREAD IMPLEMENTATION----------------------------------------------------+
import multiprocessing
import threading
import time
start = time.time()
noof_swarms = 4
num_particles=1000
proc = []
for i in range(noof_swarms):
    pp = threading.Thread(target = play, args=(num_particles,))
    #pp = threading.Thread(target = play, args=(1500,1500))
    proc.append(pp)
for k in proc:
    k.start()
    #print("process:",k)
for k in proc:
    k.join()
end = time.time()
ev2 = end - start
print("4 threads, 4 swarms:",ev2,"seconds")

4 threads, 4 swarms: 13.574320793151855 seconds


We can clearly see that for the same execution, time taken is multithreading > single core > multi core